# TP 3 : Dynamic programming applied to a knapsack problem

### Récupération des données

In [44]:
function readKnaptxtInstance(filename)
    price=[]
    weight=[]
    KnapCap=[]
    open(filename) do f
        for i in 1:3
            tok = split(readline(f))
            if(tok[1] == "ListPrices=")
                for i in 2:(length(tok)-1)
                    push!(price,parse(Int64, tok[i]))
                end
            elseif(tok[1] == "ListWeights=")
                for i in 2:(length(tok)-1)
                    push!(weight,parse(Int64, tok[i]))
                end
            elseif(tok[1] == "Capacity=")
                push!(KnapCap, parse(Int64, tok[2]))
            else
                println("Unknown read :", tok)
            end 
        end
    end
    capacity=KnapCap[1]
    return price, weight, capacity
end

readKnaptxtInstance (generic function with 1 method)

### Algorithme

In [45]:
function getSolution(C, Q, w, n)
    weight = Q
    x = zeros(n)
    # We get the same path in the reverse 
    # way to retrieve the chosen values of X
    for i=n:-1:1
        # Check if the profit was modified when X[i] was chosen, 
        # update the vector of solution X and update the current weight
        if C[i+1, weight+1] != C[i, weight+1]
            x[i] = 1
            weight -= w[i];
        else
            x[i] = 0
        end
    end
    return x
end

getSolution (generic function with 1 method)

In [52]:
function solveKnapsackDynamic(filename)

    v, w, Q = readKnaptxtInstance(filename) # Read the file
    n = length(v) # Number of items
    C = zeros(n+1,Q+1) # Profit of items (cost) - Solution
    
    # For each object
    for i=1:n
        # For each value of capacity
        for j=1:Q+1
            # Check if the weight of the current item 
            # is smaller than the remaining capacity
            if w[i] < j
                
                # We define these variables to avoid redundant calculations
                t1 = C[i, j]
                t2 = C[i, j-w[i]] + v[i] # That's why we use the if-else structure of control
                
                # If the current profit is better, update the solution
                if t1 < t2
                    C[i+1, j] = t2
                else
                    C[i+1, j] = C[i, j]
                end
            
            # Else keep the same solution for the next iteration
            else
                C[i+1, j] = C[i, j]
            end
        end
    end
    
    x = getSolution(C, Q, w, n)
    
    # The function returns the vector of solutions and the value of the objective function
    return x, C[n+1, Q+1]

    
end


solveKnapsackDynamic (generic function with 1 method)

### Questions

#### Donner une courte argumentation de l'adéquation du résultat avec l'instance résolue

Il faut vérifier que 1) la valeur de l'élément à la dernière ligne et la dernière colonne de la matrice des coûts (C ici) est égale à la valeur de la fonction-objectif (indiquée dans les noms des fichiers tests), et que 2) la somme des poids des éléments sélectionnés est inférieure à la capacité totale.

Par exemple, sur l'exemple de base, on vérifie bien qu'on a C[n+1, Q+1] == 65.0 avec 65 la valeur de la fonction-objectif et sum(x .* w) = x[2]*w[2] + x[4]*w[4] = 4.0 + 5.0 == 9.0 <= 10.0 avec 10 la capacité maximale.

In [53]:
x, C = solveKnapsackDynamic("instancesETU/KNAPnewformat/test.opb.txt")

([0.0, 1.0, 0.0, 1.0], 65.0)

#### Expliquer le fonctionnement de votre algorithme

Dans un premier temps, nous définissons un tableau de coûts C dont la taille est définie sur n+1 et Q+1, où n est le nombre d’objets disponibles et Q est la capacité totale. Pour tout i dans [1, n+1] et pour tout j dans [1, Q+1], C[i,j] représentera en fait le coût maximal des objets que l’on peut transporter si le poids maximal permis est j et que les objets que l’on peut inclure sont ceux numérotés de 1 à i. 
Ensuite, d'une part, nous savons que pour tout i dans [1, n+1], C[i,0] = 0 et d'autre part nous avons une relation de récurrence (donnée dans le sujet) qui permet, en connaissant la solution d'un problème de relativement petite taille de connaître la solution d'un problème d'une taille relativement un peu plus grande.
Ainsi, par principe de récurrence, nous pouvons compléter toute la matrice C, afin d'arriver à connaître la valeur de l'élément C[n+1, Q+1] qui correspond à la solution du problème à résoudre.

Pour des détails  plus concrets de l'implantation nous vous invitons à consulter les commentaires du code.

#### Tester l'algorithme sur pluseurs instances de tailles différentes fournies dans le TP2, donner les solutions obtenues et les valeurs de fonction-objectif. Comparer avec celles obtenues avec le branch-and-bound implémenté lors des TP2

En comparant les résultats des tests effectués à l’aide de la programmation dynamique et de l’algorithme branch-and-bound, on constate que l’utilisation de la programmation dynamique est plus rapide. Alors que pour l’algorithme branch-and-bound utilisant le borne 1, le temps d’exécution était de 111,98 secondes et en utilisant le borne 2 était de 90,57 secondes, pour les mêmes problèmes en utilisant la programmation dynamique, le temps d’exécution était de 3,22 secondes. En outre, l’allocation de mémoire a également été inférieure (65,85 M pour la programmation dynamique et 1,45 G et 1,30 G pour les bornes 1 et 2 en utilisant respectivement la branche et la branche).

Avec cela, nous pouvons conclure que la programmation dynamique semble être beaucoup plus efficace pour le problème du sac à dos que l'algorithme du branch-and-bound. 

Enfin, la question demandant les valeurs de la fonction objectif et les vecteurs solutions, nous avons fait le choix de ne pas modifier le code du TP2 (sur l'algorithme du branch-and-bound) et de ne fournir ces valeurs que pour la programmation dynamique (c'est après tout précisément ce dernier algorithme qui fait l'objet du present TP).

In [54]:
for (root, dirs, files) in walkdir("instancesETU/KNAPnewformat/test_comparision")
    @time begin
        for file in files
            x, C = solveKnapsackDynamic(joinpath.(root, file))
            println("File: ", file)
            println("Solution X: ", x)
            println("Solution Cout: ", C)
            println(" ")
        end
    end
end

File: knapPI_11_20_1000_1_-1428.opb.txt
Solution X: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Solution Cout: 1428.0
 
File: knapPI_11_50_1000_1_-1428.opb.txt
Solution X: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Solution Cout: 1428.0
 
File: knapPI_12_20_1000_1_-970.opb.txt
Solution X: [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Solution Cout: 970.0
 
File: knapPI_12_50_1000_1_-970.opb.txt
Solution X: [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Solution Cout: 970.

File: knapPI_9_100_1000_1_-995.opb.txt
Solution X: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Solution Cout: 995.0
 
File: knapPI_9_50_1000_1_-995.opb.txt
Solution X: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Solution Cout: 995.0
 
  3.184042 seconds (65.85 M allocations: 1.349 GiB, 12.00% gc time)
